In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
files = os.listdir("data/folder_1_results")
dct = {}
for file in files:
    if any(char == "7" for char in file):
        df = pd.read_csv(f"data/folder_1_results/{file}", index_col=0)
        df["id"] = df["trace_id"].astype(str) + "_" + df["ano_id"].astype(str)
        dct[file.split(".")[0] + "." + file.split(".")[1]] = df

In [ ]:
for item, df in dct.items():
    print(item, df["exp_instability"].mean())

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 3))
# axs = axs.flatten()
colors = ["red", "blue", "green", "yellow"]

idx = dct[
    "explanations_without_false_positive_filtering_without_cluster_threshold_0.7"
]["id"]
ind = np.arange(len(idx))
width = 0.2
shift = 0
items = list(dct.items())
for i in range(4):
    if i % 4 == 0:
        shift = 0
    name, df = items[i]
    color = colors[i]
    plt.bar(
        ind + shift,
        df["exp_instability"],
        # color=color,
        width=width,
        label=name,
    )
    plt.title(name)
    # ax.set_xticklabels(df["id"])
    shift += 0.2

plt.xticks(ind + width, idx, rotation=90)
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
# axs = axs.flatten()
colors = ["red", "blue", "green", "yellow"]

idx = dct[
    "explanations_without_false_positive_filtering_without_cluster_threshold_0.7"
]["id"]
ind = np.arange(len(idx))
width = 0.2
shift = 0
items = list(dct.items())
for i in range(4):
    if i % 4 == 0:
        shift = 0
    name, df = items[i]
    color = colors[i]
    plt.bar(
        ind + shift,
        df["exp_size"],
        # color=color,
        width=width,
        label=name,
    )
    plt.title(name)
    # ax.set_xticklabels(df["id"])
    shift += 0.2

plt.xticks(ind + width, idx, rotation=90)
plt.legend()
plt.show()